In [1]:
import os, sys
parent_dir = os.path.abspath('..')
# the parent_dir could already be there if the kernel was not restarted,
# and we run this cell again
if parent_dir not in sys.path:
    sys.path.append(parent_dir)


In [2]:
#module imports
import numpy as np
import pandas as pd
import NNLib.nn as nn
import NNLib.optim as optim



In [3]:
#load dataset csv
#load dataset csv
train_data = pd.read_csv('../Datasets/MNIST/mnist_train.csv')
test_data = pd.read_csv('../Datasets/MNIST/mnist_test.csv')

#train data
x_train = train_data.drop('label',axis=1).to_numpy()
y_train = pd.get_dummies(train_data['label']).values

#test data
x_test = test_data.drop('label',axis=1).to_numpy()
y_test = pd.get_dummies(test_data['label']).values

fac = 0.99 / 255
x_train = np.asfarray(x_train) * fac + 0.01
x_test = np.asfarray(x_test) * fac + 0.01


print(np.shape(x_train))
print(np.shape(y_train))


print(np.shape(x_test))
print(np.shape(y_test))


(60000, 784)
(60000, 10)
(10000, 784)
(10000, 10)


In [7]:
# defining the neural network

class DenseNN(nn.Module):
    def __init__(self, d0, d1, d2, d3):
        self.Layers = self.Sequential(
            [
                nn.Linear(d0, d1),
                nn.Sigmoid(),
                nn.Linear(d1, d2),
                nn.Sigmoid(),
                nn.Linear(d2, d3),
                nn.Softmax(),
            ]
        )

    def forward(self, z0):
        o = self.Layers.forward(z0)
        return o

In [8]:
network = DenseNN(784, 128, 64, 10)
criterion = nn.MSE(network.parameters())
optimizer = optim.SGD(network.parameters(), lr=0.01)


def compute_accuracy():
    predictions = []
    for i in range(10000):
        idx = i
        a0 = x_test[idx]
        a0 = np.array([a0]).T
        o = y_test[idx]
        o = np.array([o]).T
        output = network.forward(a0)
        pred = np.argmax(output)
        predictions.append(pred == np.argmax(o))
    return np.mean(predictions)


def train(epochs, x_train, y_train, optimizer):
    for epoch in range(epochs):
        running_loss = 0
        for i in range(60000):
            a0 = np.array([x_train[i]]).T
            y_true = np.array([y_train[i]]).T
            y_pred = network.forward(a0)
            loss = criterion(y_pred, y_true)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(
            "epoch number :",
            str(epoch),
            ", accuracy ",
            str((compute_accuracy()) * 100),
            "%",
            ", loss : ", str(running_loss/60000)
        )

In [9]:
train(10,x_train,y_train,optimizer)

epoch number : 0 , accuracy  55.44 % , loss :  [0.08000135]
epoch number : 1 , accuracy  70.36 % , loss :  [0.06164555]
epoch number : 2 , accuracy  74.46000000000001 % , loss :  [0.05148975]
epoch number : 3 , accuracy  76.5 % , loss :  [0.04537647]


KeyboardInterrupt: 